In [ ]:
#!pip uninstall pandas
#!pip install pandas
!pip install pretty_html_table

In [ ]:
import pandas as pd
import os
import re
import numpy
import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt
from pretty_html_table import build_table
import IPython



%matplotlib inline

In [ ]:

#Cargo el contenido del subconjunto del dataset que va desde Enero de 2017 hasta 31 de Agosto de 2023 incluido
train  = pd.read_excel("/content/train.xlsx")
print(len(train))

#cambio el tipo de campo a string
train.service_code = train.service_code.astype(str)

#añado campos de fecha para facilitar el análisis posterior
train["requested_datetime"] = train["requested_datetime"].apply(lambda x: x[0:10] )
train["fecha"] =  pd.to_datetime(train["requested_datetime"], format='%Y-%m-%d')
train["anyo"] = train['fecha'].dt.year


output= build_table(train.head(), 'blue_light')
IPython.display.HTML(output)
print(len(train))


40594
40594


In [ ]:
reclasif = train.groupby(["service_name","service_code"], as_index = False).size().sort_values("size", ascending = False).head(10)
output= build_table(reclasif , 'blue_light')
IPython.display.HTML(output)

service_name,service_code,size
Policia Local,4849666,3398
Limpieza viaria,250,2435
Movilidad Urbana: Señalización,1000007,2372
Calzada,60,1971
Urbanismo,5144587,1681
Contenedores,220,1667
Parques y Jardines,4849671,1604
"Césped, arbustos, flor",180,1598
Alumbrado Publico,5144577,1512
Arbolado,170,1429


In [ ]:
#Cargo el dataset de servicios ofrecidos en el desplegable del formulario del registro de quejas del Ayuntamiento
service_ayto=  pd.read_excel("/content/service_ayto.xlsx")

print(len(service_ayto))

#cambio el tipo para hacer un left join con train
service_ayto.subcodigo = service_ayto.subcodigo.astype('int64').astype('str')

output= build_table(service_ayto.head(10), 'blue_light')
IPython.display.HTML(output)


84


Unnamed: 0,codigo,subcodigo,categoria,subcategoria
0,24182784,24182784,Accesibilidad web,Accesibilidad web
1,82182145,82182145,Administración Electrónica:Web,Administración Electrónica:Web
2,5144577,5144577,Alumbrado Publico,Alumbrado Publico
3,5144577,98,Alumbrado Publico,Cables colgando
4,5144577,96,Alumbrado Publico,Columna inclinada
5,5144577,95,Alumbrado Publico,Columna oxidada y agujereada
6,5144577,94,Alumbrado Publico,Defecto en arqueta
7,5144577,93,Alumbrado Publico,Falta tape de arqueta
8,5144577,90,Alumbrado Publico,Lámpara apagada
9,5144577,91,Alumbrado Publico,Lámpara intermitente


In [ ]:

'''
A partir de aqui lo que voy a hacer es reclasificar todos los service_code, de tal manera que aquellos que en el conjunto de datos
original en realidad forman parte de una categoría superior segúnn la clasificación del desplegable de la página del ayto, queden englobados en ésta.

Creo un dataframe con el join entre los dataframes originales para averiguar que registros no tienen asignado en su columna
service_code uno de los ofrecidos en el desplegable. Son registros huérfanos, trato de agrupar los registros en base al criterio
de la página web para facilitar el estudio y la aplicación de modelos al dataset
'''

train_service = pd.merge(train, service_ayto, how = "left", left_on = "service_code", right_on ="subcodigo")

#compruebo que tengo el mismo numero de registros en el dataset original y el generado por left join
print(len(train))
print(len(train_service))

#divido el nuevo dataset "left join", en dos: según sus registros presenten o no un service_code  existente entre los contenidos en el desplegable web.
sin_codigo = train_service[train_service.codigo.isna()]
con_codigo = train_service[~train_service.codigo.isna()]
print(len(sin_codigo))
print(len(con_codigo))
#compruebo la longitud de los dataframes resultantes
print(len(sin_codigo)+ len(con_codigo) )

40594
40594
10593
30001
40594


In [ ]:
#aqui llevo el dataframe a una hoja de cálculo para su reclasificación manual
sin_codigo.to_excel("sin_codigo.xlsx")

#creo este archivo para analizar el contenido
con_codigo.to_excel("con_codigo.xlsx")

In [ ]:
'''
Cargo la  excel con la reasignación manual de códigos ("service_code").
En esta hoja de cálculo he reasignado los códigos de las categorías a las subcategorías
que aparecían como tal en el dataset
'''



servicios_reclasificados = pd.read_excel("/content/servicios_reclasificados.xlsx")
print(servicios_reclasificados)
output= build_table(servicios_reclasificados, 'blue_light')
IPython.display.HTML(output)

     service_code                 service_name  nuevo_codigo  \
0         1000003                Tapa circular       1000000   
1         1000013             Poda de arbustos       4849671   
2         1000019                        Hojas       4849671   
3         1000021              Agradecimientos       4849670   
4         1000023  Manchas de aceites o grasas       4849672   
..            ...                          ...           ...   
135            80                         Tapa       1000000   
136            81                        Falta       1000000   
137            82                  Hacer ruido       1000000   
138            83                      Hundida       1000000   
139            92                Tramo apagado       5144577   

                  categoria  
0    Ecociudad: saneamiento  
1        Parques y Jardines  
2        Parques y Jardines  
3         Buzón del Alcalde  
4          Limpieza Pública  
..                      ...  
135  Ecociudad: sanea

service_code,service_name,nuevo_codigo,categoria
1000003,Tapa circular,1000000,Ecociudad: saneamiento
1000013,Poda de arbustos,4849671,Parques y Jardines
1000019,Hojas,4849671,Parques y Jardines
1000021,Agradecimientos,4849670,Buzón del Alcalde
1000023,Manchas de aceites o grasas,4849672,Limpieza Pública
1000024,Espacios privados,4849672,Limpieza Pública
1000031,Contenedor lleno,4849672,Limpieza Pública
1000032,Excrementos de aves en el pavimento,4849672,Limpieza Pública
1000033,Hierbas en el pavimento,4849672,Limpieza Pública
106954752,Calidad de contenidos,4784129,Web Municipal


In [ ]:
#echo un vistazo a los tipos para asegurar el merge posterior
print(sin_codigo.service_code.dtypes)
print(servicios_reclasificados.service_code.dtypes)
#cmabio el tipo a string
servicios_reclasificados.service_code =servicios_reclasificados.service_code.astype('int64').astype('str')
print(servicios_reclasificados.service_code.dtypes)
#hago el join
con_codigox = pd.merge(sin_codigo, servicios_reclasificados, how = "left", on = "service_code")

#me aseguro de tener los mismos registros
print(len(sin_codigo))
print(len(con_codigox))
#compruebo que no haya ningún registro sin el nuevo service_asignado
print(con_codigox.nuevo_codigo.isna().sum())
con_codigox.to_excel("con_codigos.xlsx")
#aqui guardo los valores anteriores a la actualización para usos posteriores
con_codigox.rename(columns={'service_code': 'service_code_ant', 'service_name_x': 'service_name_ant'}, inplace = True)
#cambio el nombre de las columnas para homogeinizar el dataframe respecto al original
con_codigox.rename(columns={'nuevo_codigo': 'service_code', 'categoria_y': 'service_name'}, inplace = True)



#concateno el dataframe original con service_code correcto y el dataframe creado con los service_code corregidos
con_codigo['service_code_ant'] = con_codigo.service_code
con_codigo['service_name_ant'] = con_codigo.service_name
con_codigo['service_code'] = con_codigo.codigo
con_codigo['service_name'] = con_codigo.categoria



train_reclasificado =  pd.concat([con_codigox, con_codigo])
train_reclasificado.service_code =  train_reclasificado.service_code.astype('int64').astype(str)




train_reclasificado.to_excel("/content/train_reclasificado.xlsx")
train_reclasificado.to_csv("/content/train_reclasificado.csv")




object
int64
object
10593
10593
0


<ipython-input-33-2f7b15b77de7>:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  con_codigo['service_code_ant'] = con_codigo.service_code
<ipython-input-33-2f7b15b77de7>:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  con_codigo['service_name_ant'] = con_codigo.service_name
<ipython-input-33-2f7b15b77de7>:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pan

In [ ]:
#obtenemos el número de registros por departamento después de la reclasificación manual
train_reclasificado.service_code.value_counts()
reclasif = train_reclasificado.groupby(["service_name","service_code"], as_index = False).size().sort_values("size", ascending = False).head(10)
output= build_table(reclasif , 'blue_light')
IPython.display.HTML(output)

service_name,service_code,size
Parques y Jardines,4849671,9285
Limpieza Pública,4849672,5850
Infraestructuras:Conservacion,4849669,4747
Policia Local,4849666,3398
Alumbrado Publico,5144577,2785
Movilidad Urbana: Señalización,1000007,2372
Urbanismo,5144587,1682
Movilidad Urbana. Transporte Público,97550336,1278
Instituto Mpal. Salud Publica,5144579,847
Cultura,9043968,816
